In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit


# Load data

In [2]:
data_gauss = pd.read_csv('/Users/hkromer/02_PhD/df_MCNP_results_gauss.csv', index_col=0)
data_tophat = pd.read_csv('/Users/hkromer/02_PhD/df_MCNP_results_radial.csv', index_col=0)
df_fit = pd.read_csv('/Users/hkromer/02_PhD/02_Data/09.NeutronEmittingSpot/05.new_chamber/experiment_fit.csv', index_col=0)
df_exp = pd.read_csv('/Users/hkromer/02_PhD/02_Data/09.NeutronEmittingSpot/05.new_chamber/experiment.csv', index_col=0)

# Compute FWHM

In [3]:
def get_esf_fit(x, y):
    """
    Computes the fit to the esf (logistic fit). Needs the x and y values. Returns the fit
    """
    def fitfunc(x, *p):
        a, b, c, d = p
        z = np.exp( -( (x-b)/(c) ) )
        # z = np.exp( -( (x-p[1])/(p[2]) ) )
        return (a / ( 1 + z )) + d
        # return (p[0] / ( 1 + z )) + p[3]
    m = [1e-2, 5e-2, 1e-1, 5e-1, 0, 1, 5, 1e1]
    p0 = [1,1,1,1]
    popt, pcov = curve_fit(fitfunc, x, y, p0=p0, maxfev=1000000)
    # print(popt)
    FWHM = 3.53*popt[2]  # 3.53 * c in fermi function
    # print(FWHM)
    X_fit = np.linspace(-10, 10, 1000)
    Y_fit = fitfunc(X_fit, *popt)
    
    return X_fit, Y_fit, FWHM

In [4]:
X_fit, Y_fit, FWHM_experiment = get_esf_fit(df_exp["X"].values, df_exp["Y"].values)
FWHM_experiment

1.5384787016413026

# Find closest simulation to the experimental one

In [5]:
def scale_vals(array, vals_norm):
    
    # vals_norm = 10
    norm_cps = array[:-vals_norm].mean()
    array = array / norm_cps

    return array

## Gauss

In [6]:
fwhms = []
rads = []

colname = 'entering_tracks'

for rad in data_gauss['radius'].unique():
    df = data_gauss.loc[ data_gauss['radius'] == rad ]
    df[colname] = scale_vals(df[colname].values, vals_norm=10)
    X_fit, Y_fit, FWHM = get_esf_fit(df['x_pos'].values, df[colname].values)
    fwhms.append(FWHM)
    rads.append(rad)
fwhms = np.asarray(fwhms, dtype=float)
rads = np.asarray(rads, dtype=float)

<ipython-input-6-c00ce3d5b676>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname] = scale_vals(df[colname].values, vals_norm=10)


In [7]:
# closest value
idx = np.argmin(np.abs(fwhms-FWHM_experiment))
fwhms[idx], rads[idx]

(1.5208637246476335, 0.17)

## Tophat

In [8]:
fwhms = []
rads = []

colname = 'entering_tracks'

for rad in data_tophat['diameter'].unique():
    df = data_tophat.loc[ data_tophat['diameter'] == rad ]
    df[colname] = scale_vals(df[colname].values, vals_norm=10)
    X_fit, Y_fit, FWHM = get_esf_fit(df['x_pos'].values, df[colname].values)
    fwhms.append(FWHM)
    rads.append(rad)
    
fwhms = np.asarray(fwhms, dtype=float)
rads = np.asarray(rads, dtype=float)
# closest value
idx = np.argmin(np.abs(fwhms-FWHM_experiment))
fwhms[idx], rads[idx]

<ipython-input-8-c867625bdd31>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colname] = scale_vals(df[colname].values, vals_norm=10)


(1.52313252478264, 2.6)

In [14]:
# See above
val_gauss = 0.17 # fwhm in cm

val_tophat = 2.6 # diameter in mm


data_gauss_0 = data_gauss.loc[ data_gauss['radius'] == val_gauss ]
data_gauss_0 = data_gauss_0.sort_values(by='x_pos')


data_tophat_0 = data_tophat.loc[ data_tophat['diameter'] == val_tophat ]
data_tophat_0 = data_tophat_0.sort_values(by='x_pos')


In [15]:
%matplotlib widget

data_gauss_1 = data_gauss_0.copy()
data_tophat_1 = data_tophat_0.copy()

vals_norm = 10
norm_cps = data_gauss_1['entering_tracks'].values[:-vals_norm].mean()
data_gauss_1['entering_tracks'] = data_gauss_1['entering_tracks'].values / norm_cps

norm_cps = data_tophat_1['entering_tracks'].values[:-vals_norm].mean()
data_tophat_1['entering_tracks'] = data_tophat_1['entering_tracks'].values / norm_cps

scale_cps = data_gauss_1['entering_tracks'].values[0:vals_norm].mean()
data_gauss_1['entering_tracks'] = data_gauss_1['entering_tracks'].values - scale_cps

scale_cps = data_tophat_1['entering_tracks'].values[0:vals_norm].mean()
data_tophat_1['entering_tracks'] = data_tophat_1['entering_tracks'].values - scale_cps

# HEAD
sns.set(context='paper', font_scale=1.5)
fig, ax = plt.subplots(figsize=(8,6))

# GAUSS
X = data_gauss_1['x_pos'].values
Y = data_gauss_1['entering_tracks'].values

# plt.plot(X, Y, marker='o', markersize=2, color='darkblue', label='Gauss')
plt.plot(X, Y, color='darkblue', label='Gauss', lw=2.5)


# TOPHAT
X = data_tophat_1['x_pos'].values
Y = data_tophat_1['entering_tracks'].values

plt.plot(X, Y,color='darkorange', label='Tophat', lw=2.5)


# LAYOUT
plt.ylabel('Simulated entering tracks in detector (normalized) [-]')
plt.xlabel('Edge position [mm]')
plt.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Experiment

In [16]:
%matplotlib widget

plt.plot(df_fit['X_fit'].values, df_fit['Y_fit'].values, color='red', label='Logistic fit')
lbl = 'Experiment'
plt.scatter(df_exp["X"].values, df_exp["Y"].values, marker='o', color='blue', label=lbl, s=40)
plt.errorbar(df_exp["X"].values, df_exp["Y"].values, xerr=None, yerr=df_exp["Y_err"].values, ls='none', color='blue', capsize=5)
plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Both - unscaled

In [30]:
%matplotlib widget

# scale the simulation to the experiment
def rescale_linear(array, new_min, new_max):
    """Rescale an arrary linearly."""
    minimum, maximum = np.min(array), np.max(array)
    m = (new_max - new_min) / (maximum - minimum)
    b = new_min - m * minimum
    return m * array + b
    
# maximum, minimum from the logistic fit
exp_max = np.max(df_fit['Y_fit'].values)
exp_min = np.min(df_fit['Y_fit'].values)

data_gauss_1 = data_gauss_0.copy()
data_tophat_1 = data_tophat_0.copy()

vals_norm = 10
norm_cps = data_gauss_1['entering_tracks'].values[:-vals_norm].mean()
data_gauss_1['entering_tracks'] = data_gauss_1['entering_tracks'].values / norm_cps

norm_cps = data_tophat_1['entering_tracks'].values[:-vals_norm].mean()
data_tophat_1['entering_tracks'] = data_tophat_1['entering_tracks'].values / norm_cps

scale_cps = data_gauss_1['entering_tracks'].values[0:vals_norm].mean()
data_gauss_1['entering_tracks'] = data_gauss_1['entering_tracks'].values - scale_cps

scale_cps = data_tophat_1['entering_tracks'].values[0:vals_norm].mean()
data_tophat_1['entering_tracks'] = data_tophat_1['entering_tracks'].values - scale_cps



# HEAD
sns.set(context='paper', font_scale=1.5)
fig, ax = plt.subplots(figsize=(8,6))

# GAUSS
X = data_gauss_1['x_pos'].values
Y = data_gauss_1['entering_tracks'].values

# plt.plot(X, Y, marker='o', markersize=2, color='darkblue', label='Gauss')
plt.plot(X, Y, color='darkblue', label='Gauss', ls='--', lw=2.5, alpha=0.7)


# TOPHAT
X = data_tophat_1['x_pos'].values
Y = data_tophat_1['entering_tracks'].values

plt.plot(X, Y,color='darkorange', label='Tophat', ls='--', lw=2.5, alpha=0.7)

# EXPERIMENT
plt.plot(df_fit['X_fit'].values, df_fit['Y_fit'].values, color='red', label='Logistic fit', lw=2)
lbl = 'Experiment'
plt.scatter(df_exp["X"].values, df_exp["Y"].values, marker='o', color='blue', label=lbl, s=40, alpha=0.3)
plt.errorbar(df_exp["X"].values, df_exp["Y"].values, xerr=None, yerr=df_exp["Y_err"].values, ls='none', color='blue', capsize=5, alpha=0.1)
plt.legend()
plt.show()

# LAYOUT
plt.ylabel('Normalized counts in detector [-]')
plt.xlabel('Edge position [mm]')
plt.legend()
plt.xlim([-6, 6])
plt.ylim([-0.1, 0.9])
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Both - scaled

In [29]:
%matplotlib widget

# scale the simulation to the experiment
def rescale_linear(array, new_min, new_max):
    """Rescale an arrary linearly."""
    minimum, maximum = np.min(array), np.max(array)
    m = (new_max - new_min) / (maximum - minimum)
    b = new_min - m * minimum
    return m * array + b
    
# maximum, minimum from the logistic fit
exp_max = np.max(df_fit['Y_fit'].values)
exp_min = np.min(df_fit['Y_fit'].values)

data_gauss_1 = data_gauss_0.copy()
data_tophat_1 = data_tophat_0.copy()

vals_norm = 10
norm_cps = data_gauss_1['entering_tracks'].values[:-vals_norm].mean()
data_gauss_1['entering_tracks'] = data_gauss_1['entering_tracks'].values / norm_cps

norm_cps = data_tophat_1['entering_tracks'].values[:-vals_norm].mean()
data_tophat_1['entering_tracks'] = data_tophat_1['entering_tracks'].values / norm_cps

scale_cps = data_gauss_1['entering_tracks'].values[0:vals_norm].mean()
data_gauss_1['entering_tracks'] = data_gauss_1['entering_tracks'].values - scale_cps

scale_cps = data_tophat_1['entering_tracks'].values[0:vals_norm].mean()
data_tophat_1['entering_tracks'] = data_tophat_1['entering_tracks'].values - scale_cps


# scale to experiment
data_tophat_1['entering_tracks_scaled'] = rescale_linear(data_tophat_1['entering_tracks'].values, exp_min, exp_max)
data_gauss_1['entering_tracks_scaled'] = rescale_linear(data_gauss_1['entering_tracks'].values, exp_min, exp_max)

# HEAD
sns.set(context='paper', font_scale=1.5)
fig, ax = plt.subplots(figsize=(8,6))

# GAUSS
X = data_gauss_1['x_pos'].values
Y = data_gauss_1['entering_tracks_scaled'].values

# plt.plot(X, Y, marker='o', markersize=2, color='darkblue', label='Gauss')
plt.plot(X, Y, color='darkblue', label='Gauss', ls='--', lw=2.5, alpha=0.7)


# TOPHAT
X = data_tophat_1['x_pos'].values
Y = data_tophat_1['entering_tracks_scaled'].values

plt.plot(X, Y,color='darkorange', label='Tophat', ls='--', lw=2.5, alpha=0.7)

# EXPERIMENT
plt.plot(df_fit['X_fit'].values, df_fit['Y_fit'].values, color='red', label='Logistic fit', lw=2)
lbl = 'Experiment'
plt.scatter(df_exp["X"].values, df_exp["Y"].values, marker='o', color='blue', label=lbl, s=40, alpha=0.3)
plt.errorbar(df_exp["X"].values, df_exp["Y"].values, xerr=None, yerr=df_exp["Y_err"].values, ls='none', color='blue', capsize=5, alpha=0.1)
plt.legend()
plt.show()

# LAYOUT
plt.ylabel('Normalized and scaled counts in detector [-]')
plt.xlabel('Edge position [mm]')
plt.legend()
plt.xlim([-6, 6])
plt.ylim([-0.1, 0.7])
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …